Yahoo Finance API pull from the 15 biggest pharmaseutical companies (By MCap)

In [ ]:
import yfinance as yf

tickers = [
    "LLY", "JNJ", "ABBV", "RHHBY", "AZN", "NVS", "NVO", "MRK",
    "AMGN", "PFE", "GILD", "SNY", "BMY", "GSK", "BIIB", "OGN"
]

company_names = {
    "LLY": "Eli Lilly and Company",
    "JNJ": "Johnson & Johnson",
    "ABBV": "AbbVie Inc.",
    "RHHBY": "Roche Holding AG",
    "AZN": "AstraZeneca PLC",
    "NVS": "Novartis AG",
    "NVO": "Novo Nordisk A/S",
    "MRK": "Merck & Co., Inc.",
    "AMGN": "Amgen Inc.",
    "PFE": "Pfizer Inc.",
    "GILD": "Gilead Sciences, Inc.",
    "SNY": "Sanofi",
    "BMY": "Bristol-Myers Squibb",
    "GSK": "GlaxoSmithKline plc",
    "BIIB": "Biogen Inc.",
    "OGN": "Organon & Co."
}

start = datetime.datetime(2015, 1, 1)
end   = datetime.datetime(2025, 1, 1)
out_dir = "by_ticker_csv"

def yf_long(tickers, startdate, enddate, interval="1d", auto_adjust=False, progress=False):
    """
    Returns DataFrame:
    Date, Ticker, Open, High, Low, Close, Adj Close, Volume
    """
    df = yf.download(
        tickers=tickers,
        start=startdate,
        end=enddate,
        interval=interval,
        auto_adjust=auto_adjust,
        progress=progress,
        group_by="column"
    )

    if isinstance(df.columns, pd.MultiIndex):
        if any(t in df.columns.levels[0] for t in tickers):
            df_long = df.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index()
        else:
            df_long = df.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()
    else:
        df_long = df.reset_index()
        df_long["Ticker"] = tickers if isinstance(tickers, str) else tickers[0]

    df_long = df_long[["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]]
    df_long["Date"] = pd.to_datetime(df_long["Date"])
    df_long = df_long.sort_values(["Ticker", "Date"]).reset_index(drop=True)
    return df_long

data = yf_long(tickers, start, end)

shares_out = {}
for tkr in tickers:
    try:
        info = yf.Ticker(tkr).info
        shares_out[tkr] = info.get("sharesOutstanding", None)
    except Exception:
        shares_out[tkr] = None

os.makedirs(out_dir, exist_ok=True)

for i, tkr in enumerate(sorted(data["Ticker"].unique()), start=1):
    df_t = data[data["Ticker"] == tkr].copy().sort_values("Date")

    df_t["Name"] = company_names.get(tkr, tkr)

    shares = shares_out.get(tkr)
    if shares is not None:
        df_t["Marketcap"] = df_t["Close"] * shares
    else:
        df_t["Marketcap"] = None
    
    df_t.insert(0, "SNo", df_t.index + 1)

    df_t = df_t[["SNo", "Name", "Ticker", "Date", "High", "Low", "Open", "Close", "Volume", "Marketcap"]]

    out_path = os.path.join(out_dir, f"{tkr}.csv")
    df_t.to_csv(out_path, index=False, float_format="%.4f", date_format="%Y-%m-%d")
    print(f"Wrote {out_path}")